# Preprocessing: Water treatment plants

**Objective**: Prepare data from the **MERKUR** dataset for use in machine learning algorithms.

**Background**: The MERKUR project, based in the *Research Centre for Built Environment, Climate, Water Technology and Digitalisation* at VIA University College, collects and analyzes data from water treatment plants in Denmark. In short, the project aims to understand how water treatment plants are run, and the results are then ideally used to optimize the running of water treatment plants. However, the dataset is, as of now, relatively "dirty" in a machine learning context: There are many missing values, outliers, a mix of categorical and numeric data, etc.

**Data Source**: The dataset has kindly been provided to us by Senior Associate Professor Loren Mark Ramsay. You can read more [here](https://en.via.dk/research/built-environment-climate-water-technology-and-digitalisation/water-treatment-and-distribution) and [here](https://www.ucviden.dk/en/projects/merkur-national-web-baseret-dataplatform-til-drikkevandsbehandlin).

Note that we are only working with a subset of the full database. This subset is saved as an Excel file, `merkur.xlsx`.

#### Overall Instructions
1. Explore the dataset to understand the features and their distributions.
2. Preprocess the data, handling any missing values, outliers, etc.

Below some suggestions are given but the assignment is relatively "free".

Best of luck with your analysis!

In [51]:
import pandas as pd

# Assuming you have already read the Excel file into a DataFrame
raw_data = pd.read_excel(r"merkur.xlsx")

# - Overview of missing values in the dataset
missing_values = raw_data.isnull().sum()

# - Percentage of missing values in the dataset
missing_percentage = (missing_values / len(raw_data)) * 100

# * Create a new dataframe with the missing values
missing_summary = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage, 'Data Type': raw_data.dtypes})
missing_summary = missing_summary.sort_values(by='Percentage', ascending=False)
display(missing_summary)

,Missing Values,Percentage,Data Type
UniformityCoefficient,51,64.556962,float64
SumOfld_layer,39,49.367089,float64
TotalEBCT,39,49.367089,float64
AverageFilterBedVolume,34,43.037975,float64
OverallFilterGrainSizeMin,34,43.037975,float64
OverallFilterGrainSizeMax,34,43.037975,float64
AverageTotalFilterDepth,33,41.772152,float64
TankExploitation,30,37.974684,float64
HLR_BW,27,34.177215,float64
Stage1HLR,21,26.582278,float64


## __Fill missing variables:__

### *Numerical Values:*
We fill all missing numerical values with the median of the current column values. This is partly because using the median value is resistant to outliers which helps the preservation of the central tendancy.

### *Categorical Values:*
We fill all missing categorical values with the mode of the current column. The reason we do this is because even categorical values can be filled using the mode value of a speciffic column, since it is the most common value present in the column. Another reason is the preservation of the distribution within the dataset. Imputing using the mode variable helps maintain the distribution of the categorical variable.

In [52]:
# ! Extract the columns with more than 40% missing values
columns_to_drop = missing_summary[missing_summary['Percentage'] > 40]

data_cleaned = raw_data.copy()
data_cleaned = data_cleaned.drop(columns=columns_to_drop.index)
data_cleaned = data_cleaned.drop(columns=['WaterworksName'])

# * Impute missin g values for numerical columns using the median
numerical_columns = data_cleaned.select_dtypes(include=['float64','int64']).columns
data_cleaned[numerical_columns] = data_cleaned[numerical_columns].fillna(data_cleaned[numerical_columns].median())

# * Impute missing values for categorical columns using the mode
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

# Fill missing data with mdoe value of current col
for column in categorical_columns:
    mode_value = data_cleaned[column].mode()[0]
    data_cleaned[column] = data_cleaned[column].fillna(mode_value)
    
display(data_cleaned)

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,FilterExploitation,AerationType,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,6,50.00,6.000000,5200.00000,10.00,Time,57.886530,Cascade,1.880000,1.388889,143.341204,866.666667,0.414525,30.889453,27.294778,19.402985,Gravity,Single,253543
1,4,110.00,4.908739,300.00000,28.00,Volume,63.521586,Air injection,1.620000,11.204508,16.714412,40.743665,23.000062,30.557749,2.576075,19.402985,Pressure,Double,612094
2,4,120.00,15.343900,3800.00000,83.30,Volume,64.908486,Bottom aeration,2.213333,3.935020,44.693822,249.217919,2.192086,32.791831,21.825600,19.402985,Gravity,Double,682318
3,2,120.00,16.000000,1200.00000,26.00,Volume,63.279585,Cascade,2.073333,3.742502,194.588021,75.000000,2.166907,30.889453,12.554300,19.402985,Gravity,Single,156998
4,2,120.00,1.495000,250.00000,4.50,Volume,63.279585,Cascade,1.313333,3.742502,412.300024,167.224093,1.818182,30.889453,52.220566,19.402985,Gravity,Single,6710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,16,1426.00,47.940002,3250.00000,110.00,Mix,91.698648,Cascade,1.546667,5.242647,24.106636,258.637091,0.178441,30.889453,9.176959,39.682540,Gravity,Double,11454775
75,20,1407.29,10.178760,4500.00000,57.00,Volume,89.405620,Passive plate aerator,2.133333,6.912876,22.050800,442.097088,1.236822,51.086775,6.040393,19.402985,Pressure,Single,11021800
76,6,250.00,14.752500,2331.50137,51.57,Time,19.331598,Bottom aeration,1.673333,2.824380,209.076387,158.041107,4.446077,30.367735,22.367525,45.833333,Gravity,Single,423362
77,12,120.00,37.437401,4400.00000,134.00,Volume,63.279585,Cascade,2.200000,3.742502,152.513970,93.489396,35.386015,32.053507,15.097670,19.402985,Gravity,Double,1521895


In [53]:
data_cleaned.isna().sum()

TotalFilters               0
MaxTypicalFlow             0
AverageFilterArea          0
AverageTypicalRunVolume    0
AverageBackwashVolume      0
PrimaryTrigger             0
FilterExploitation         0
AerationType               0
OxygenFactor               0
Stage1HLR                  0
Footprint                  0
UFRV                       0
BW%                        0
HLR_BW                     0
TankCapacity               0
TankExploitation           0
GravityPressureMixed       0
Stages                     0
AbstractedVolume           0
dtype: int64

## Removing the outliers

The next section aims to remove the outliers present in the current data set.

We remove a row if it has an outlier in any column. <-----

In [54]:
import pandas as pd
from scipy import stats
import numpy as np

irrelevant_columns = ['TotalFilters', 'TankCapacity', 'OxygenFactor']
relevant_columns = [col for col in numerical_columns if col not in irrelevant_columns]

# Calculate the z-score for all values in relevant columns
for col in relevant_columns:
    z_scores = stats.zscore(data_cleaned[col])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3)  # Adjust 3 as per requirement
    data_cleaned = data_cleaned[filtered_entries]

# Reset index
data_cleaned.reset_index(drop=False, inplace=True)

# Now, data_cleaned has only the filtered entries with a new continuous index

## Scaling the data

This section aims to scale our data, since unscaled data can seem extreme in the difference between data points.

We use MinMaxScaling here because <-----

In [72]:
from sklearn.preprocessing import MinMaxScaler

irrelevant_columns = ['index']
relevant_columns = [col for col in numerical_columns if col not in irrelevant_columns]

# Declare 
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data_cleaned[relevant_columns])
scaled_df = pd.DataFrame(scaled, columns=relevant_columns)
scaled_df['Org Index'] = data_cleaned['index']

display(scaled_df)

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,AbstractedVolume,Org Index
0,0.294118,0.062464,0.193906,0.225012,0.100029,0.393157,0.610390,0.038377,0.672509,0.973378,0.026832,0.587067,0.261953,0.337714,0.064922,0
1,0.176471,0.162981,0.581657,0.160430,0.896999,0.464762,0.880952,0.206473,0.181051,0.233672,0.166777,0.637074,0.203570,0.337714,0.197452,2
2,0.058824,0.162981,0.608883,0.040491,0.273992,0.448152,0.767316,0.193763,0.927819,0.024958,0.164795,0.587067,0.104600,0.337714,0.035081,3
3,0.647059,0.299397,0.538337,0.115837,0.317483,0.518609,0.534632,0.070759,0.389525,0.113133,0.105345,0.234644,0.253538,0.155996,0.395153,5
4,0.235294,0.063900,0.110910,0.025268,0.105465,0.216348,0.226190,0.788443,0.059187,0.039933,0.313066,0.587067,0.022205,0.662818,0.042547,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.176471,0.096927,0.050518,0.019732,0.121774,0.323206,1.000000,0.906625,0.045868,0.170501,0.114320,0.808087,0.239154,0.200129,0.088794,71
58,0.411765,0.162981,0.957465,0.248077,0.991592,0.448152,0.870130,0.193763,0.513519,0.151142,1.000000,0.550758,0.221795,0.337714,0.262692,72
59,0.176471,0.107094,0.266527,0.140132,0.160916,0.063710,0.788961,0.119357,0.340822,0.454501,0.135205,0.587067,1.000000,0.122257,0.042710,73
60,0.294118,0.349655,0.557115,0.092687,0.552008,0.000000,0.442641,0.133149,1.000000,0.124442,0.344232,0.573353,0.209355,0.864503,0.117412,76


Here we combine our scaled data with the categorical columns on the cleaned data.

In [73]:
# combine the scaled dataframe with the original categorical columns
combined_df = pd.concat([scaled_df, data_cleaned[categorical_columns]], axis=1)

# One hot encode the combined dataframe
one_hot_encoded_df = pd.get_dummies(combined_df, columns=categorical_columns)

# Utility function - Set the column order to be more intuitive
def reorder_columns(data : pd.DataFrame) -> pd.DataFrame:
    # ? Reordering the columns
    # So the columns are easier to understand and work with.
    strict_column_order = ['Org Index']
    remaining_columns = [col for col in data.columns if col not in strict_column_order]
    data = data[strict_column_order + remaining_columns]
    return data

one_hot_encoded_df = reorder_columns(one_hot_encoded_df)

display(one_hot_encoded_df)

,Org Index,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,...,AerationType_Other,AerationType_Passive plate aerator,AerationType_Pure oxygen injection,GravityPressureMixed_Gravity,GravityPressureMixed_Mixed,GravityPressureMixed_Pressure,Stages_Double,Stages_Mixed,Stages_Single,Stages_Triple
0,0,0.294118,0.062464,0.193906,0.225012,0.100029,0.393157,0.610390,0.038377,0.672509,...,False,False,False,True,False,False,False,False,True,False
1,2,0.176471,0.162981,0.581657,0.160430,0.896999,0.464762,0.880952,0.206473,0.181051,...,False,False,False,True,False,False,True,False,False,False
2,3,0.058824,0.162981,0.608883,0.040491,0.273992,0.448152,0.767316,0.193763,0.927819,...,False,False,False,True,False,False,False,False,True,False
3,5,0.647059,0.299397,0.538337,0.115837,0.317483,0.518609,0.534632,0.070759,0.389525,...,False,False,False,True,False,False,True,False,False,False
4,6,0.235294,0.063900,0.110910,0.025268,0.105465,0.216348,0.226190,0.788443,0.059187,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,71,0.176471,0.096927,0.050518,0.019732,0.121774,0.323206,1.000000,0.906625,0.045868,...,False,False,False,False,False,True,True,False,False,False
58,72,0.411765,0.162981,0.957465,0.248077,0.991592,0.448152,0.870130,0.193763,0.513519,...,False,False,False,True,False,False,True,False,False,False
59,73,0.176471,0.107094,0.266527,0.140132,0.160916,0.063710,0.788961,0.119357,0.340822,...,False,False,False,True,False,False,False,False,True,False
60,76,0.294118,0.349655,0.557115,0.092687,0.552008,0.000000,0.442641,0.133149,1.000000,...,False,False,False,True,False,False,False,False,True,False


### Suggestions

-  Filter out (i.e. remove) any irrelevant columns (e.g. names, IDs, etc.)
-  Several columns contain missing values (NaNs). Find out how large a percentage each column is missing. Perhaps some of them lack so much data that you should consider removing them?
-  Scale numeric data.
-  For the features you choose to keep, impute the missing values in an appropriate way - or perhaps you find it more appropriate to delete the rows?
-  Several features (e.g., "PrimaryTrigger") are categorical. Use one-hot encoding to turn them into numeric data. Be careful with the feature "Stages" - perhaps one-hot encoding is not the best choice here?
-  If you you choose to remove or replace outliers, do this now. If you choose to keep, move on.
-  Create a correlation matrix and discuss - based on this, you might want to drop certain columns.
-  Consider whether some features should be transformed (e.g. using log, square root etc.) and do this if found relevant.
-  There are only about 80 rows in the data set. Discuss consequences of this in terms of machine learning - as well as potential solutions. 
-  Think about whether there are other steps you find appropriate at this point. If not, declare your data set clean.